<a href="https://colab.research.google.com/github/Mahalaxmikukudala/Machine-learning-basic-predictions/blob/main/predictive_maintainance.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
 import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from itertools import combinations
from sklearn.naive_bayes import GaussianNB, BernoulliNB
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score
from sklearn.preprocessing import MinMaxScaler
import matplotlib.pyplot as plt
from sklearn.metrics import accuracy_score, confusion_matrix

In [ ]:
df=pd.read_csv("engine_data.csv")

In [ ]:
df.head()

,Engine rpm,Lub oil pressure,Fuel pressure,Coolant pressure,lub oil temp,Coolant temp,Engine Condition
0,700,2.493592,11.790927,3.178981,84.144163,81.632187,1
1,876,2.941606,16.193866,2.464504,77.640934,82.445724,0
2,520,2.961746,6.553147,1.064347,77.752266,79.645777,1
3,473,3.707835,19.510172,3.727455,74.129907,71.774629,1
4,619,5.672919,15.738871,2.052251,78.396989,87.000225,0


In [ ]:
df.isnull().sum()

Engine rpm          0
Lub oil pressure    0
Fuel pressure       0
Coolant pressure    0
lub oil temp        0
Coolant temp        0
Engine Condition    0
dtype: int64

In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 19535 entries, 0 to 19534
Data columns (total 7 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   Engine rpm        19535 non-null  int64  
 1   Lub oil pressure  19535 non-null  float64
 2   Fuel pressure     19535 non-null  float64
 3   Coolant pressure  19535 non-null  float64
 4   lub oil temp      19535 non-null  float64
 5   Coolant temp      19535 non-null  float64
 6   Engine Condition  19535 non-null  int64  
dtypes: float64(5), int64(2)
memory usage: 1.0 MB


In [ ]:
df.duplicated().sum()

0

In [ ]:
df['Engine Condition'].value_counts()

Engine Condition
1    12317
0     7218
Name: count, dtype: int64

In [ ]:
df=df[df['Coolant temp']<=100]
df

,Engine rpm,Lub oil pressure,Fuel pressure,Coolant pressure,lub oil temp,Coolant temp,Engine Condition
0,700,2.493592,11.790927,3.178981,84.144163,81.632187,1
1,876,2.941606,16.193866,2.464504,77.640934,82.445724,0
2,520,2.961746,6.553147,1.064347,77.752266,79.645777,1
3,473,3.707835,19.510172,3.727455,74.129907,71.774629,1
4,619,5.672919,15.738871,2.052251,78.396989,87.000225,0
...,...,...,...,...,...,...,...
19530,902,4.117296,4.981360,4.346564,75.951627,87.925087,1
19531,694,4.817720,10.866701,6.186689,75.281430,74.928459,1
19532,684,2.673344,4.927376,1.903572,76.844940,86.337345,1
19533,696,3.094163,8.291816,1.221729,77.179693,73.624396,1


In [ ]:
df = df[(df['Engine rpm'] <= 1500) & (df['Engine rpm'] >= 400)]

In [ ]:
corr_matrix = df.corr()
corr_matrix["Engine Condition"].sort_values(ascending=False)

Engine Condition    1.000000
Fuel pressure       0.116564
Lub oil pressure    0.062867
Coolant pressure   -0.025299
Coolant temp       -0.047292
lub oil temp       -0.095158
Engine rpm         -0.254801
Name: Engine Condition, dtype: float64

In [ ]:
target = 'Engine Condition'
X = df.loc[:,df.columns!=target]
y = df.loc[:,df.columns==target]

In [ ]:
scaler = MinMaxScaler()
X = scaler.fit_transform(X)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [ ]:
df["Coolant Efficiency"] = (1 / df["Engine rpm"]) * df["Coolant temp"]
df["Oil Efficiency"] = 1 / (df["Engine rpm"] * df["lub oil temp"])
df['pressure_coeff'] = df['Lub oil pressure']*df['Coolant pressure']
df['temp_coeff'] = df['lub oil temp']*df['Coolant temp']
df['coolant_coeff'] = df['Coolant pressure']*df['Coolant temp']
df['lub_coeff'] = df['Lub oil pressure']*df['lub oil temp']


<ipython-input-19-d26a31f0bb4a>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df["Coolant Efficiency"] = (1 / df["Engine rpm"]) * df["Coolant temp"]
<ipython-input-19-d26a31f0bb4a>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df["Oil Efficiency"] = 1 / (df["Engine rpm"] * df["lub oil temp"])
<ipython-input-19-d26a31f0bb4a>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the docum

In [ ]:
df = df[(df['temp_coeff'] <= 7200) & (df['temp_coeff'] >= 5000)]

In [ ]:
df = df[(df['coolant_coeff'] <= 400) & (df['coolant_coeff'] >= 70)]


In [ ]:
df = df[(df['lub_coeff'] <= 430) & (df['lub_coeff'] >= 120)]


In [ ]:
target = 'Engine Condition'
X = df.loc[:,df.columns!=target]
y = df.loc[:,df.columns==target]

In [ ]:
from imblearn.over_sampling import SMOTE

oversample = SMOTE()

X1, y1 = oversample.fit_resample(X, y)
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
X1_scaled = scaler.fit_transform(X1)
X_train, X_test, y_train, y_test = train_test_split(X1, y1, test_size=0.2, random_state=42)

In [ ]:
print(y1[target].value_counts())

Engine Condition
1    10505
0    10505
Name: count, dtype: int64


In [ ]:
def filterout(col, x1, x2):
    column_index = col
    X1_filtered = [row for row in X1_scaled if x1 <= row[column_index] <= x2]
    return X1_filtered

In [ ]:
y_train = np.asarray(y_train).astype('float32').reshape((-1,1))
y_test1 = np.asarray(y_test).astype('float32').reshape((-1,1))

In [ ]:
model = RandomForestClassifier(n_estimators=200, random_state=42)
model.fit(X_train, y_train)

predictions = model.predict(X_test)
accuracy = accuracy_score(y_test, predictions)

print(f"Final Classification Accuracy: {accuracy}")


<ipython-input-29-8bea0a4bd865>:2: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  model.fit(X_train, y_train)


Final Classification Accuracy: 0.6899095668729177


In [ ]:
model = LogisticRegression()
model.fit(X_train, y_train)

predictions = model.predict(X_test)
accuracy = accuracy_score(y_test, predictions)

print(f"Final Classification Accuracy: {accuracy}")


/usr/local/lib/python3.10/dist-packages/sklearn/utils/validation.py:1143: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


Final Classification Accuracy: 0.6211327939076631


In [ ]:
model = DecisionTreeClassifier(random_state=42)
model.fit(X_train, y_train)

predictions = model.predict(X_test)
accuracy = accuracy_score(y_test, predictions)

print(f"Final Classification Accuracy: {accuracy}")

Final Classification Accuracy: 0.6118514992860543


In [ ]:
from sklearn.svm import SVC

In [ ]:
model = SVC(kernel="linear",random_state=42)
model.fit(X_train, y_train)

predictions = model.predict(X_test)
accuracy = accuracy_score(y_test, predictions)

print(f"Final Classification Accuracy: {accuracy}")

/usr/local/lib/python3.10/dist-packages/sklearn/utils/validation.py:1143: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


Final Classification Accuracy: 0.6211327939076631


In [ ]:
from sklearn.naive_bayes import GaussianNB, BernoulliNB
gnb=GaussianNB()
gnb.fit(X_train,y_train)
predictions=gnb.predict(X_test)
accuracy=accuracy_score(y_test,predictions)
print(f"Final Classification Accuracy: {accuracy}")

Final Classification Accuracy: 0.6168491194669206


/usr/local/lib/python3.10/dist-packages/sklearn/utils/validation.py:1143: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
